# PHASE 2 PROJECT: Movie Studio Analysis

## INTRODUCTION
### BUSINESS PROBLEM 
Your company now sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of your company's new movie studio can use to help decide what type of films to create.

### OBJECTIVES
In this project we will be looking to give out a clear picture of what the movie industry is like with regards to profits, popularity of genres and when the best season to release a movie is so as to maximize profits.
To acomplish this we will be using concepts learnt in SQL, Statistical methods and even Hypothesis testing

## DATA UNDERSTANDING
For this project we will be having 6 data sources:
1. `bom.movie_gross.csv`: This is a file that contains information regarding a movies name, both domestic and foreign gross amount and the year it was released

2. `im.db`:This is a zipped folder that contains a database called `im` which contains data regarding a movie. This data includes what category it falls under, the direcctor, actors involved, movie ratings, running time and many more

3. `rt.movie_info.tsv`: This file has information the movie rating for theatrical releases e.g. 'R' 'PG 13' e.t.c. it also tells more about the movie like genre, director, theatrical release date, dvd release date, runtime, box office gross and studio

4. `rt.reviews.tsv`: This is a list of reviews from reviewers and and the ratings they gave, it also gives info on who published the review

5. `tmdb.movies.csv`: Here we have genre ids, movie titles, language the movie is in, its popularity, release date,vote count and the average vote

6. `tn.movie_budgets.csv`: This file contains a movies production budget, domestic gross and worlwide gross

## DATA PREPARATION

Now we want to take a closer look at the data sources above

In [15]:
#We start by importing the relevant libraries that we would use to explore the data sources and come to a conclussion
import pandas as pd
import sqlite3 
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import io

Now we want to load the data sources below

In [16]:
# Here we are loading `bom.movie_gross.csv``
mg = pd.read_csv(r"C:\Users\User\Documents\DS-LABS\Projects\Phase2Project\zippedData\bom.movie_gross.csv.gz").head()
mg

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010


In [17]:
# Loading 'rt.movie_info.tsv'
mi = pd.read_csv(r"zippedData\rt.movie_info.tsv.gz",  sep = '\t', encoding='ISO-8859-1').head()
mi

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [18]:
# Load and view `rt.reviews.tsv`
mr = pd.read_csv(r"zippedData\rt.reviews.tsv.gz", sep = '\t', encoding='ISO-8859-1').head()
mr

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [19]:
#Load 'tn.movie_budgets.csv'
mb = pd.read_csv(r"zippedData\tn.movie_budgets.csv.gz").head()
mb

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [20]:
#Here we want to unzip the zipped database file
zipped_db = r"zippedData\im.db.zip"
db_file = r"zippedData\im.db"

with zipfile.ZipFile(zipped_db, 'r') as zip_ref:
    if db_file in zip_ref.namelist():
        with zip_ref.open(db_file) as db:
            db_buffer = io.BytesIO(db_file.read())

mdb = sqlite3.connect(db_file).cursor()
mdb.execute("SELECT name FROM sqlite_master WHERE type = 'table';").fetchall()#This querry will display tables from the database 'im'

[('movie_basics',),
 ('directors',),
 ('known_for',),
 ('movie_akas',),
 ('movie_ratings',),
 ('persons',),
 ('principals',),
 ('writers',)]

Now for the next section we want to get a closer look at all the tables from the database we have loaded above

In [21]:
mdb.execute(f"PRAGMA table_info('movie_basics');").fetchall()

[(0, 'movie_id', 'TEXT', 0, None, 0),
 (1, 'primary_title', 'TEXT', 0, None, 0),
 (2, 'original_title', 'TEXT', 0, None, 0),
 (3, 'start_year', 'INTEGER', 0, None, 0),
 (4, 'runtime_minutes', 'REAL', 0, None, 0),
 (5, 'genres', 'TEXT', 0, None, 0)]

In [22]:
mdb.execute(f"PRAGMA table_info('directors');").fetchall()

[(0, 'movie_id', 'TEXT', 0, None, 0), (1, 'person_id', 'TEXT', 0, None, 0)]

In [23]:
mdb.execute(f"PRAGMA table_info('known_for');").fetchall()

[(0, 'person_id', 'TEXT', 0, None, 0), (1, 'movie_id', 'TEXT', 0, None, 0)]

In [24]:
mdb.execute(f"PRAGMA table_info('movie_akas');").fetchall()

[(0, 'movie_id', 'TEXT', 0, None, 0),
 (1, 'ordering', 'INTEGER', 0, None, 0),
 (2, 'title', 'TEXT', 0, None, 0),
 (3, 'region', 'TEXT', 0, None, 0),
 (4, 'language', 'TEXT', 0, None, 0),
 (5, 'types', 'TEXT', 0, None, 0),
 (6, 'attributes', 'TEXT', 0, None, 0),
 (7, 'is_original_title', 'REAL', 0, None, 0)]

In [25]:
mdb.execute(f"PRAGMA table_info('movie_ratings');").fetchall()

[(0, 'movie_id', 'TEXT', 0, None, 0),
 (1, 'averagerating', 'REAL', 0, None, 0),
 (2, 'numvotes', 'INTEGER', 0, None, 0)]

In [26]:
mdb.execute(f"PRAGMA table_info('persons');").fetchall()

[(0, 'person_id', 'TEXT', 0, None, 0),
 (1, 'primary_name', 'TEXT', 0, None, 0),
 (2, 'birth_year', 'REAL', 0, None, 0),
 (3, 'death_year', 'REAL', 0, None, 0),
 (4, 'primary_profession', 'TEXT', 0, None, 0)]

In [27]:
mdb.execute(f"PRAGMA table_info('principals');").fetchall()

[(0, 'movie_id', 'TEXT', 0, None, 0),
 (1, 'ordering', 'INTEGER', 0, None, 0),
 (2, 'person_id', 'TEXT', 0, None, 0),
 (3, 'category', 'TEXT', 0, None, 0),
 (4, 'job', 'TEXT', 0, None, 0),
 (5, 'characters', 'TEXT', 0, None, 0)]

In [28]:
mdb.execute(f"PRAGMA table_info('writers');").fetchall()

[(0, 'movie_id', 'TEXT', 0, None, 0), (1, 'person_id', 'TEXT', 0, None, 0)]

This will now make it easier to find the columns that we would be using to connect two or more of the tables together. Now that we've seen what they contain and the data types of the columns. Now the task ahead will be to try ad find a way of matching the database extracted to `mdb` with the other files in order to be able to analyse the data and come out insights that could help the company make the decision on what kind of movie studio they want to opn.

So the best way to look at the kind of data that would be needed would be trying to answer the following questions
1. What is the most popular movie genre among customers?
2. What is the total profit made by movies depending on the genres?
3. when is the most appropriate time to release a movie in order to make the most amount of profit?

1. What is the most popular movie genre among customers?

Now in order to answer this we would want to look at the ratings that are given to the movies and now the 